# Load My Spotify Data
### Plays

In [1]:
import json
from tqdm.notebook import tqdm

In [2]:
musics_my_data = {}
artists_my_data = {}

In [3]:
# Opening JSON file
f = open('./Datasets/4. My Spotify Data/MyData/StreamingHistory0.json', encoding="utf8")
  
# returns JSON object as 
# a dictionary
data = json.load(f)

In [4]:
limit = -1

for item in data:
    # Remove Podcasts
    if 'podcast' in item['artistName'].lower():
        continue
        
    # Limit
    if limit != -1:
        if limit == 0:
            break
        limit = limit - 1
    
    # Variables
    trackName = item['trackName']
    artist = item['artistName']
    track_id = (trackName + artist).lower()
    
    # Music
    if track_id not in musics_my_data:
        musics_my_data[track_id] = {}
    musics_my_data[track_id]['trackName'] = trackName
    musics_my_data[track_id]['artistName'] = artist
    musics_my_data[track_id]['stream'] = True
    
    # Artist
    if artist not in artists_my_data:
        artists_my_data[artist] = {}
        artists_my_data[artist]['musics'] = []
    artists_my_data[artist]['musics'].append(track_id)

In [5]:
print('First Step: Streaming ')
print('Number of songs: ' + str(len(musics_my_data)))
print('Number of artists: ' + str(len(artists_my_data)))

First Step: Streaming 
Number of songs: 2696
Number of artists: 1459


### Playlists "Novas"

In [6]:
# Opening JSON file
f = open('./Datasets/4. My Spotify Data/MyData/Playlist1.json', encoding="utf8")
  
# returns JSON object as 
# a dictionary
data = json.load(f)

In [7]:
favorites_playlists = {
    'Novas 2017': 2017, 
    'Novas 2018': 2018,
    'Novas 2019': 2019,
    'Novas 2020': 2020,
    'Novas 2021': 2021
}

limit = -1

for playlist in data['playlists']:
    if playlist['name'] not in favorites_playlists:
        continue
    playlist_year = favorites_playlists[playlist['name']]
    for item in playlist['items']:
        # Limit
        if limit != -1:
            if limit == 0:
                break
            limit = limit - 1

        # Variables
        trackName = item['track']['trackName']
        artist = item['track']['artistName']
        track_id = (trackName + artist).lower()

        # Music
        if track_id not in musics_my_data:
            musics_my_data[track_id] = {}
        musics_my_data[track_id]['trackName'] = trackName
        musics_my_data[track_id]['artistName'] = artist
        musics_my_data[track_id]['playlist'] = playlist_year

        # Artist
        if artist not in artists_my_data:
            artists_my_data[artist] = {}
            artists_my_data[artist]['musics'] = []
        artists_my_data[artist]['musics'].append(track_id)

In [8]:
#print(json.dumps(musics_my_data, indent=4))
print('Second Step: Streaming + Playlists ')
print('Number of songs: ' + str(len(musics_my_data)))
print('Number of artists: ' + str(len(artists_my_data)))

Second Step: Streaming + Playlists 
Number of songs: 2748
Number of artists: 1469


---
# Spotify

In [9]:
%env SPOTIPY_CLIENT_ID=79739c5371df43a4b955af5f24560590
%env SPOTIPY_CLIENT_SECRET=84f4caa9d1154f0696a3290fca96438e
%env SPOTIPY_REDIRECT_URI=http://localhost

env: SPOTIPY_CLIENT_ID=79739c5371df43a4b955af5f24560590
env: SPOTIPY_CLIENT_SECRET=84f4caa9d1154f0696a3290fca96438e
env: SPOTIPY_REDIRECT_URI=http://localhost


In [10]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials

spotify = spotipy.Spotify(client_credentials_manager=SpotifyClientCredentials())

### Music Search

In [11]:
spotify_music_file = open('./Datasets/spotify_music_json.json', 'a', encoding="utf8")

count = 0

spotify_music_file.write('[\n')
for music in tqdm(musics_my_data):
    # Search Track
    results = {}
    try:
        results = spotify.search(q='track:' + musics_my_data[music]['trackName'] + ' ' +  musics_my_data[music]['artistName'], type='track')
    except:
        pass
        
    try:
        selectedTrack = ''
        for track in results['tracks']['items']:
            if 'BR' in track['available_markets']:
                #selectedTrack = track
                break
        if selectedTrack == '':
            selectedTrack = results['tracks']['items'][0]
    except:
        musics_my_data[music]['spotifyID'] = '-'
        artists_my_data[musics_my_data[music]['artistName']]['spotifyID'] = '-'
        print(music + ' failed!')
        count = count + 1
        continue
        
    # Get Spotify ID
    musics_my_data[music]['spotifyID'] = selectedTrack['id']
    musics_my_data[music]['spotifyName'] = selectedTrack['name']
    musics_my_data[music]['spotifyArtistName'] = selectedTrack['artists'][0]['name']
    artists_my_data[musics_my_data[music]['artistName']]['spotifyID'] = selectedTrack['artists'][0]['id']
    
    spotify_music_file.write(json.dumps(selectedTrack))
    
    count = count + 1
    if count != len(musics_my_data):
        spotify_music_file.write(',\n')
    
spotify_music_file.write(']') 
spotify_music_file.close()

psychotica (vs doktor mohlberg)bootleg rascal failed!
see you again - one world: together at homecharlie puth failed!
what a wonderful world - one world: together at homecamila cabello failed!
stand by me - one world: together at homejohn legend failed!
i will never let you down - one world: together at homerita ora failed!
despacito - one world: together at homeluis fonsi failed!
rainbow - one world: together at homekesha failed!
the bones - one world: together at homehozier failed!
for you - one world: together at homerita ora failed!
attention - one world: together at homecharlie puth failed!
mad world - one world: together at homeadam lambert failed!
flashlight - one world: together at homejessie j failed!
mr. brightside - one world: together at homethe killers failed!
hit the road jack (remastered)ray charles failed!
bang bang - one world: together at homejessie j failed!
l'amour toujoursmayra failed!
you're my everything - original version with false startmiles davis quintet fail

### Music Features

In [12]:
features_musics = []
count_n = 100
for i in tqdm(range(0, len(musics_my_data), count_n)):
    i_min = i;
    i_max = i + count_n
    if i_max > len(musics_my_data):
        i_max = len(musics_my_data)
    keys_to_run = list(musics_my_data.keys())[i_min:i_max]
    
    ids_to_run = []
    for k in keys_to_run:
        if musics_my_data[k].get('spotifyID', '-') != '-':
            ids_to_run.append(musics_my_data[k]['spotifyID'])
        
    features_musics.extend(spotify.audio_features(tracks=ids_to_run))
    
spotify_features_file = open('./Datasets/spotify_features_json.json', 'a', encoding="utf8")
spotify_features_file.write(json.dumps(features_musics))
spotify_features_file.close()

### Artists

In [13]:
artists_music_spotify = []
count_n = 50
for i in tqdm(range(0, len(artists_my_data), count_n)):
    i_min = i;
    i_max = i + count_n
    if i_max > len(artists_my_data):
        i_max = len(artists_my_data)
    keys_to_run = list(artists_my_data.keys())[i_min:i_max]
    
    ids_to_run = []
    for k in keys_to_run:
        if artists_my_data[k].get('spotifyID', '-') != '-':
            ids_to_run.append(artists_my_data[k]['spotifyID'])
        
    try:
        artists_music_spotify.extend(spotify.artists(ids_to_run)['artists'])
    except:
        pass
    
spotify_artists_file = open('./Datasets/spotify_artists_json.json', 'a', encoding="utf8")
spotify_artists_file.write(json.dumps(artists_music_spotify))
spotify_artists_file.close()

### Artists Albums
To see if he is still active and when was his last launch

---
# Genius

In [15]:
import lyricsgenius

In [16]:
genius = lyricsgenius.Genius('SpcMrm5O-CPXVGJ3uMC_U01aB07BCQVdfKkhK9vW-0G5NGDgSzmORalh4idCwCWR')

### Get songs + Lyrics info

In [17]:
genius_songs = {}
count = 0

genius_file = open('./Datasets/genius_json.json', 'a', encoding="utf8")
genius_lyrics_file = open('./Datasets/genius_lyrics_json.json', 'a', encoding="utf8")
genius_file.write('[\n')
genius_lyrics_file.write('[\n')
for music in tqdm(musics_my_data):
    count = count + 1
    try:
        song = genius.search_song(musics_my_data[music]['trackName'], musics_my_data[music]['artistName'])
        genius_file.write(song.to_json())

        # Get info
        musics_my_data[music]['geniusID'] = song.to_dict()['id']
        musics_my_data[music]['geniusName'] = song.to_dict()['title']
        musics_my_data[music]['geniusArtistName'] = song.to_dict()['primary_artist']['name']
        artists_my_data[musics_my_data[music]['artistName']]['geniusID'] = song.to_dict()['primary_artist']['id']

        # Get Lyrics
        lyrics = {
            'geniusID': song.to_dict()['id']
        }
        try:
            lyrics['lyrics'] = song.lyrics
        except:
            lyrics['lyrics'] = 'N/A'

        genius_lyrics_file.write(json.dumps(lyrics))
        if count != len(musics_my_data):
            genius_file.write(',\n')
            genius_lyrics_file.write(',\n')
    except:
        pass

genius_file.write(']') 
genius_file.close()
genius_lyrics_file.write(']') 
genius_lyrics_file.close()

Searching for "Spain" by Chick Corea...
Done.
Searching for "Singular" by ANAVITÓRIA...
Done.
Searching for "Sutilmente" by Skank...
Done.
Searching for "Dangerous (feat. Joywave) - Oliver Remix" by Big Data...
Done.
Searching for "Freight train" by Elizabeth Cotten...
Done.
Searching for "Take Me To Church" by Hozier...
Done.
Searching for "Amianto" by Supercombo...
Done.
Searching for "I Bet My Life" by Imagine Dragons...
Done.
Searching for "Hey Jude" by The Beatles...
Done.
Searching for "Come Together - Remastered 2009" by The Beatles...
Done.
Searching for "O Sol" by Vitor Kley...
Done.
Searching for "IDGAF" by Dua Lipa...
Done.
Searching for "Born To Be Wild" by Steppenwolf...
Done.
Searching for "Jolene" by Dolly Parton...
Done.
Searching for "Hurt" by Johnny Cash...
Done.
Searching for "Bulls On Parade" by Rage Against The Machine...
Done.
Searching for "Black Magic Woman" by Fleetwood Mac...
Done.
Searching for "Heart of Gold - 2009 Remaster" by Neil Young...
No results found

No results found for: 'Ensaio Sobre A Incerteza Boca'
Searching for "Outra Vez - Ao Vivo" by Saulo...
Done.
Searching for "Se fosse tão fácil - Acústico" by MAR ABERTO...
No results found for: 'Se fosse tão fácil - Acústico MAR ABERTO'
Searching for "Fica Tudo Bem" by Silva...
Done.
Searching for "Apenas Mais uma de Amor" by Lulu Santos...
Done.
Searching for "Me Diz" by Ariane Villa Lobos...
No results found for: 'Me Diz Ariane Villa Lobos'
Searching for "Céu Azul (Ao Vivo)" by Charlie Brown Jr....
No results found for: 'Céu Azul (Ao Vivo) Charlie Brown Jr.'
Searching for "relicário" by ANAVITÓRIA...
Done.
Searching for "Fica" by ANAVITÓRIA...
Done.
Searching for "Praia do Rosa - Acústico" by Oriente...
Done.
Searching for "Pijama - Acústico" by Gabriel Elias...
No results found for: 'Pijama - Acústico Gabriel Elias'
Searching for "Deixa a Luz Entrar" by Paulo Novaes...
Done.
Searching for "Tempo Perdido (Versão Os Dias Eram Assim)" by Tiago Iorc...
No results found for: 'Tempo Perdid

Done.
Searching for "Myself" by Bazzi...
Done.
Searching for "Nothing Breaks Like a Heart (feat. Miley Cyrus)" by Mark Ronson...
Done.
Searching for "Hold On (feat. Cheat Codes) - Radio Edit" by Moguai...
Done.
Searching for "Cosmic Bound" by HANNIE...
Done.
Searching for "Sweater Weather" by The Neighbourhood...
Done.
Searching for "Ne reste pas de glace" by Léa Paci...
Done.
Searching for "Wherever I Go" by OneRepublic...
Done.
Searching for "Slow Dancing in a Burning Room" by John Mayer...
Done.
Searching for "Gravity" by John Mayer...
Done.
Searching for "Swift Code" by Christian Löffler...
Done.
Searching for "The Searchers" by Felix Cartal...
Done.
Searching for "Gendèr" by Makoto San...
Done.
Searching for "It Fades Away" by boerd...
Done.
Searching for "Beats Religion" by Sieeben Acht...
Done.
Searching for "Oxygen.absorber" by Austin Haga...
No results found for: 'Oxygen.absorber Austin Haga'
Searching for "Try - Interlude" by Shallou...
Done.
Searching for "Trippest Flu" by S

Done.
Searching for "Softly As In A Morning Sunrise - Remastered" by Sonny Clark Trio...
No results found for: 'Softly As In A Morning Sunrise - Remastered Sonny Clark Trio'
Searching for "Talk It Up" by Sammy Rae & The Friends...
Done.
Searching for "Almost Like Being In Love" by Red Garland...
Done.
Searching for "Three to See the King" by Jakob Bro...
Done.
Searching for "Reflections" by Kurt Rosenwinkel...
No results found for: 'Reflections Kurt Rosenwinkel'
Searching for "Bumpin' On Sunset" by Wes Montgomery...
Specified song does not contain lyrics. Rejecting.
Searching for "That Lucky Old Sun (Just Rolls Around Heaven All Day)" by Grant Green...
Done.
Searching for "Night And Day" by Django Reinhardt...
Done.
Searching for "Come Rain or Come Shine" by Ben Street...
Done.
Searching for "You Must Believe in Spring" by Mike Moreno...
Done.
Searching for "Bonde (with Ry Cooder)" by Ali Farka Touré...
No results found for: 'Bonde (with Ry Cooder) Ali Farka Touré'
Searching for "In Yo

Done.
Searching for "Blues for Alice" by Charlie Parker...
Specified song does not contain lyrics. Rejecting.
Searching for "Red House" by Jimi Hendrix...
Done.
Searching for "Green Onions" by Booker T. & the M.G.'s...
Specified song does not contain lyrics. Rejecting.
Searching for "C Jam Blues" by Oscar Peterson Trio...
Done.
Searching for "Move" by Miles Davis...
Specified song does not contain lyrics. Rejecting.
Searching for "Pharaoh's Dance (feat. John McLaughlin, Chick Corea, Wayne Shorter & Bennie Maupin)" by Miles Davis...
No results found for: 'Pharaoh's Dance (feat. John McLaughlin, Chick Corea, Wayne Shorter & Bennie Maupin) Miles Davis'
Searching for "A Love Supreme, Pt. I – Acknowledgement" by John Coltrane...
No results found for: 'A Love Supreme, Pt. I – Acknowledgement John Coltrane'
Searching for "Bitches Brew (feat. John McLaughlin, Wayne Shorter, Chick Corea & Joe Zawinul)" by Miles Davis...
No results found for: 'Bitches Brew (feat. John McLaughlin, Wayne Shorter, 

...

Done.
Searching for "Once in a While" by No Spirit...
Specified song does not contain lyrics. Rejecting.
Searching for "Fonder" by braj mahal...
Done.
Searching for "Amanheceu" by Scalene...
Done.
Searching for "Ethos" by Scalene...
Done.
Searching for "Legado" by Scalene...
Done.
Searching for "cartão postal" by Scalene...
Done.
Searching for "fragmento" by Scalene...
Done.
Searching for "Scarecrow - Radio Edit" by Clairity...
Done.
Searching for "impulso" by Scalene...
Done.
Searching for "Feed The Right Wolf" by A Beautiful Anarchy...
Done.
Searching for "Gimme Gimme" by Groove Delight...
Done.
Searching for "Apart" by KIDSØ...
Done.
Searching for "Smalltown Boy" by Bronski Beat...
Done.
Searching for "What’s Next" by Drake...
Done.
Searching for "Another Time" by TWO LANES...
Done.
Searching for "Gone Feral" by James Holden...
Done.
Searching for "First Flight" by Kaitlyn Aurelia Smith...
Done.
Searching for "In den Gärten Pharaos" by Popol Vuh...
No results found for: 'In den Gärt

Specified song does not contain lyrics. Rejecting.
Searching for "In the Androgynous Dark" by Brambles...
Done.
Searching for "Etude" by Joep Beving...
Done.
Searching for "Recomposed By Max Richter: Vivaldi, The Four Seasons: Spring 1" by Max Richter...
No results found for: 'Recomposed By Max Richter: Vivaldi, The Four Seasons: Spring 1 Max Richter'
Searching for "Dream 1 (before the wind blows it all away) - Pt. 1" by Max Richter...
Specified song does not contain lyrics. Rejecting.
Searching for "The Departure" by Max Richter...
Specified song does not contain lyrics. Rejecting.
Searching for "Spark" by Niklas Paschburg...
Specified song does not contain lyrics. Rejecting.
Searching for "On The Nature Of Daylight" by Max Richter...
Specified song does not contain lyrics. Rejecting.
Searching for "Loss" by Raffael Seyfried...
Done.
Searching for "Veils" by Raffael Seyfried...
No results found for: 'Veils Raffael Seyfried'
Searching for "Hinterhof" by Raffael Seyfried...
No results f

Done.
Searching for "The Pines" by Bill Laurance...
Done.
Searching for "Couscous" by Benny Greb...
Done.
Searching for "16 Bars" by The Funky Knuckles...
Done.
Searching for "Lymaks" by FORQ...
Done.
Searching for "Grebfruit" by Benny Greb...
Specified song does not contain lyrics. Rejecting.
Searching for "Wise Willis" by The Funky Knuckles...
Done.
Searching for "Grebchestra" by Benny Greb...
No results found for: 'Grebchestra Benny Greb'
Searching for "Taizo" by FORQ...
Done.
Searching for "Montreal" by Mark Lettieri...
Specified song does not contain lyrics. Rejecting.
Searching for "Naptime" by Mark Lettieri...
Specified song does not contain lyrics. Rejecting.
Searching for "Funky D" by Victor Wooten...
Done.
Searching for "Gigantactis" by Mark Lettieri...
Specified song does not contain lyrics. Rejecting.
Searching for "U Can't Hold No Groove (If You Ain't Got No Pocket)" by Victor Wooten...
No results found for: 'U Can't Hold No Groove (If You Ain't Got No Pocket) Victor Woote

No results found for: 'Você Não Sabe o Que é Amor - Ao Vivo Luan Santana'
Searching for "Vento Ventania" by Biquini Cavadão...
Done.
Searching for "River" by Oh, Be Clever...
Done.
Searching for "Jerome" by Zella Day...
Done.
Searching for "Californian Soil" by London Grammar...
Done.
Searching for "Lose Your Head" by London Grammar...
Done.
Searching for "Crawling Kingsnake" by The Black Keys...
Done.
Searching for "Old Friend" by Sea Wolf...
Done.
Searching for "Intro" by London Grammar...
Done.
Searching for "Missing" by London Grammar...
Done.
Searching for "Break My Heart" by Dua Lipa...
Done.
Searching for "Get Up I Feel Like Being A Sex Machine - Pt. 1 / Single Version" by James Brown...
No results found for: 'Get Up I Feel Like Being A Sex Machine - Pt. 1 / Single Version James Brown'
Searching for "We're Good" by Dua Lipa...
Done.
Searching for "Ain't No Mountain High Enough" by Marvin Gaye...
Done.
Searching for "Super Bad - Pts.1 & 2" by James Brown...
Done.
Searching for "W

Done.
Searching for "Let Me Go (with Alesso, Florida Georgia Line & watt)" by Hailee Steinfeld...
Done.
Searching for "Blood / / Water" by grandson...
Done.
Searching for "Can't Stop Me Now" by Oh The Larceny...
Done.
Searching for "Wise Enough" by Lamb...
Done.
Searching for "Com Você é Melhor" by DUX...
No results found for: 'Com Você é Melhor DUX'
Searching for "Changing" by Sigma...
Done.
Searching for "Nevada (feat. Cozi Zuehlsdorff)" by Vicetone...
Done.
Searching for "Friday Night" by Vigiland...
Done.
Searching for "Way Back (feat. Cozi Zuehlsdorff)" by Vicetone...
Done.
Searching for "Miracles (feat. Bjørnskov)" by Martin Jensen...
Done.
Searching for "Follow (feat. Sleeper)" by Lucian...
Done.
Searching for "LoveSick" by Rigby ft. Layne...
No results found for: 'LoveSick Rigby ft. Layne'
Searching for "Try Me" by Tropkillaz...
Done.
Searching for "Somewhere New (feat. M-22) - Radio Edit" by Klingande...
No results found for: 'Somewhere New (feat. M-22) - Radio Edit Klingande'

### Get Artists

In [21]:
genius_artists_file = open('./Datasets/genius_artists_json.json', 'a', encoding="utf8")

count = 0

genius_artists_file.write('[\n')
for artist in tqdm(artists_my_data):
    print(artist)
    
    geniusId = artists_my_data[artist].get('geniusID', '-')
    if geniusId == '-':
        count = count + 1
        continue
        
    artist = genius.artist(geniusId)
    genius_artists_file.write(json.dumps(artist))
    
    count = count + 1
    if count != len(artists_my_data):
        genius_artists_file.write(',\n')
    
genius_artists_file.write(']') 
genius_artists_file.close()

Chick Corea
ANAVITÓRIA
Skank
Big Data
Elizabeth Cotten
Hozier
Supercombo
Imagine Dragons
The Beatles
Vitor Kley
Dua Lipa
Steppenwolf
Dolly Parton
Johnny Cash
Rage Against The Machine
Fleetwood Mac
Neil Young
John Denver
The Revivalists
Beck
Chappell Roan
LÉON
Chloe Lilac
Zella Day
Declan McKenna
Orla Gartland
Winona Oak
Allie X
MUNA
Skott
Kailee Morgue
Cyn
Billie Marten
Ashe
VÉRITÉ
Priscilla Alcantara
Djavan
Zeca Baleiro
Alex Cohen
Time Machine
Axe BA
Raul Seixas
Eric Clapton
Of Monsters and Men
The Animals
Red Hot Chili Peppers
X Ambassadors
Léa Paci
Oasis
Bootleg Rascal
Now United
SZA
London Grammar
Chimarruts
MARINA
Ava Max
Tones And I
Jão
Two For Larry
The Black Keys
alt-J
Projota
Melim
Alan Ellis
1Kilo
Hotelo
Natiruts
Milton Nascimento
Anitta
Dilsinho
Titãs
Orgânico
Jorge & Mateus
Gabily
Sarita Lorena
Lagum
Ana Vilela
Sandy
Roberta Campos
Hungria Hip Hop
Zeeba
Barão Vermelho
Gabriel Gonti
Boca
Saulo
MAR ABERTO
Silva
Lulu Santos
Ariane Villa Lobos
Charlie Brown Jr.
Oriente
Gabriel 

Marshmello
Dido
MØ
Corinne Bailey Rae
Rootkit
Mike Posner
El Speaker
Giulia Be
ThatBehavior
K/DA
Lilac
Phlocalyst
Guustavv
Slumberville
Lofi Coffee
Fthmlss
Liquid Stranger
Zeds Dead
Little Birds
Coquins
Kiiara
Maty Noyes
Warren Haynes
Kina
League of Legends
Mike Perry
DJ Shadow
Madcon
JR JR
BRAZA
Mozella
Feint
The Constellations
The xx
Black Rebel Motorcycle Club
Jetta
Coleman Hell
Gotye
Cold War Kids
Royal & the Serpent
Skandra
Lil Wayne
Robert Jates
Thirteen Senses
AREA21
Fall Out Boy
The Blue Stones
Noah Guthrie
Alessia Cara
Katy Perry
Glass Animals
Prefekt
Tritonal
Vigiland
Tash Sultana
Barns Courtney
G-Eazy
Andrew Huang
American Authors
Ellie Goulding
Jakoban
Grizfolk
Vandaveer
grandson
Zayde Wølf
HYDDE
Massive Vibes
Kygo
Elephante
The DNC
Julia Michaels
lovelytheband
IZA
Macklemore
Sigala
Martin Garrix
Thirty Seconds To Mars
Bryce Fox
Machine Gun Kelly
Oh The Larceny
Florence + The Machine
DUX
Royal Tailor
Axwell /\ Ingrosso
Lindsey Stirling
Seinabo Sey
Cat Power
Andrew Applepie


Nick Mulvey
Ray LaMontagne
Lord Huron
The Paper Kites
Ross Copperman
Stateless
Hey Violet
Antônio Neves
Karen Souza
The Nychillharmonic
Prisoner
GoGo Penguin
Mette Henriette
Eivind Aarset
Areni Agbabian
Matthew Halsall
Joe Lovano & Dave Douglas Sound Prints
Brad Mehldau Trio
Dexter Gordon
Justin Bieber
Olivia Rodrigo
Kali Uchis
Black Eyed Peas
Duncan Laurence
JASSS
Gregory Alan Isakov
CloZee
Death Cab for Cutie
Jarrod Lawson
44th Move
Reuben James
Jamie Cullum
D'Angelo
KOKOROKO
BUNT.
Oshima Brothers
Luan Santana
Oh, Be Clever
Sea Wolf
James Brown
Marvin Gaye
Earth, Wind & Fire
Vulfpeck
INSIDE the SOUND 
Cecilie Strange
Caity Gyorgy
Smoothe Sounds
Ambiworld
Cedric Moulin
Amor de la luna
Valentina Romano
Piano Miles
Michael Silverman
Grover Washington, Jr.
Usher
Billie Holiday
Jacaszek
Floating Points
George Benson
Jim Hall
Yuri Honing Acoustic Quartet
Oded Tzur
Nestor Torres
Heavy Mellow
Hablot Brown
Farraday's Cage
Gracie Convert
Myles Jasnowski
Jill Scott
LTTLE KNG
Ms. Lauryn Hill
Jor

---
# IDs
### Save ids

In [18]:
ids_file = open('./Datasets/ids_json.json', 'a', encoding="utf8")
ids_file.write(json.dumps(musics_my_data))
ids_file.close()

ids_artists_file = open('./Datasets/ids_artists_json.json', 'a', encoding="utf8")
ids_artists_file.write(json.dumps(artists_my_data))
ids_artists_file.close()

---
# Checks
### Load ids

In [117]:
ids_file = open('./Datasets/ids_json.json', encoding="utf8")
ids_artists_file = open('./Datasets/ids_artists_json.json', encoding="utf8")
lyrics_file = open('./Datasets/genius_lyrics_json.json', encoding="utf8")

ids = json.load(ids_file)
ids_artists = json.load(ids_artists_file)
lyrics_complete = json.load(lyrics_file)
lyrics = []
for lyric in lyrics_complete:
    lyrics.append(lyric['geniusID'])

### Basic Statistics

In [118]:
total_count = len(ids)
spotify_count = 0
genius_count = 0
lyrics_count = len(lyrics)

diff_genius_count = 0
diff_spotify_count = 0

diff_genius = {}
diff_spotify = {}

for music_id in ids:
    genius_id = ids[music_id].get('geniusID', '-')
    spotify_id = ids[music_id].get('spotifyID', '-')
    trackName =  ids[music_id].get('trackName', '').lower()
    geniusName =  ids[music_id].get('geniusName', '').lower()
    spotifyName =  ids[music_id].get('spotifyName', '').lower()
    artistName =  ids[music_id].get('artistName', '').lower()
    spotifyArtistName =  ids[music_id].get('spotifyArtistName', '').lower()
    geniusArtistName =  ids[music_id].get('geniusArtistName', '').lower()
    
    # Normalizations
    trackName = trackName.replace('’', '\'')
    geniusName = geniusName.replace('’', '\'')
    spotifyName = spotifyName.replace('’', '\'')
    
    # count ocurrences
    if genius_id != '-':
        genius_count = genius_count + 1
    if spotify_id != '-':
        spotify_count = spotify_count + 1
        
    # get name
    if genius_id != '-':
        if trackName != geniusName and trackName not in geniusName and geniusName not in trackName:
            if artistName != geniusArtistName:
                diff_genius[music_id] = trackName + ' (' + artistName + ') \n\t' + geniusName + ' (' + geniusArtistName + ')'
    if spotify_id != '-':
        if trackName != spotifyName and trackName not in spotifyName and spotifyName not in trackName:
            diff_spotify[music_id] = trackName + ' (' + artistName + ') \n\t' + spotifyName + ' (' + spotifyArtistName + ')'
        
print('Total of Songs: ' + str(total_count))
print('Songs that were found on Spotify: ' + str(spotify_count))
print('Songs that were found on Genius: ' + str(genius_count))
print('Songs that have lyrics: ' + str(lyrics_count))
print('-'*30)
print('Songs that doesn\'t have the same name on Spotify: ' + str(len(diff_spotify)))
print('Songs that doesn\'t have the same name on Genius: ' + str(len(diff_genius)))

Total of Songs: 2748
Songs that were found on Spotify: 2618
Songs that were found on Genius: 1990
Songs that have lyrics: 1988
------------------------------
Songs that doesn't have the same name on Spotify: 64
Songs that doesn't have the same name on Genius: 513


### Manual Check

In [119]:
# confirmed_spotify = [1, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 60, 61, 62, 63]
# confirmed_spotify_ids = []
# for i in confirmed_spotify:
#     confirmed_spotify_ids.append(diff_spotify_keys[i])
confirmed_spotify_ids = ["you're everythingchick corea", 'goldberg variations, bwv 988: variation no. 30: quodlibetjohann sebastian bach', 'piano sonata no. 8 in a minor, k. 310: ii. andante cantabile con espressionewolfgang amadeus mozart', 'ballade no. 3 in a-flat major, op. 47frédéric chopin', 'nocturne in e-flat major, op. 55 no. 2frédéric chopin', 'waltz in d-flat major, op. 64, no. 1 "minute waltz"frédéric chopin', 'étude in c-sharp minor, op. 10, no. 4frédéric chopin', '12 etudes, op.10: no.5 in g flat "black keys"frédéric chopin', "goldberg variations, bwv 988: variation 3 a 1 clav. canone all'unisonojohann sebastian bach", 'prélude in e minor, op. 28, no. 4frédéric chopin', 'goldberg variations, bwv 988: variation 4 a 1 clav.johann sebastian bach', 'goldberg variations, bwv 988: variation 5 a 1 ovvero 2 clav.johann sebastian bach', 'morceaux de fantaisie, op. 3: no. 2 in c-sharp minor. préludesergei rachmaninoff', 'goldberg variations, bwv 988: variation 7 a 1 ovvero 2 clav.johann sebastian bach', 'etude, op. 10 no. 2 in a minor: allegrofrédéric chopin', "menuet sur le nom d'haydnmaurice ravel", "jeux d'eau, m. 30maurice ravel", 'goldberg variations, bwv 988: variation 8 a 2 clav.johann sebastian bach', 'goldberg variations, bwv 988: variation 10 a 1 clav. fughettajohann sebastian bach', 'piano sonata no.2, in g sharp minor op.19 "sonata fantasy": 1. andantealexander scriabin', 'ravel: concerto for piano left hand, m. 82maurice ravel', 'book ii: prelude and fugue no. 22 in b-flat minor bwv 891: i. preludejohann sebastian bach', 'book ii: prelude and fugue no. 22 in b-flat minor bwv 891: ii. fuguejohann sebastian bach', 'piano sonata no.2, in g sharp minor op.19 "sonata fantasy": 2. prestoalexander scriabin', "piano trio in a minor, op.50, th.117: var. i: l'istesso tempo - livepyotr ilyich tchaikovsky", 'the nutcracker, op.71, th.14: overturepyotr ilyich tchaikovsky', 'swan lake, op.20, th.12 / act 1: no.1 scène (allegro giusto)pyotr ilyich tchaikovsky', 'unaccompanied cello suite no. 3 in c major, bwv 1009: préludejohann sebastian bach', 'unaccompanied cello suite no. 3 in c major, bwv 1009: bourrée i & bourrée iijohann sebastian bach', "the four seasons: 'spring' (concerto in e op.8 no.1): allegroantonio vivaldi", 'ii. hungarian rhapsodyfranz liszt', 'quer ir no meu show?hyperanhas', 'beaux dimanches - liveamadou & mariam', 'beggin (original version)madcon', 'dance me to the end of love - bonusthe civil wars', 'cardiac arrest (woodysproduce remix)bad suns', 'piano sonata no. 14 in c-sharp minor, op. 27, no. 2, "moonlight": i. adagio sostenutoludwig van beethoven', '3 gymnopédies: no. 1, lent et douloureuxerik satie', 'piano sonata no. 8 in c minor, op. 13, "pathétique": ii. adagio cantabileludwig van beethoven', 'symphony no. 5 in c minor, op. 67: i. allegro con brioludwig van beethoven', 'canon in d major, p.37johann pachelbel', 'piano concerto no. 2 in c minor, op. 18: 2. adagio sostenutosergei rachmaninoff', 'the piano: the heart asks pleasure firstmichael nyman', 'piano concerto no. 2 in c minor, op. 18: 1. moderatosergei rachmaninoff', 'sonata no. 23 "appassionata" in f minor, op. 57: i. allegro assailudwig van beethoven', "the soldier's marchrobert schumann", 'introductory movement - portrait versionyann tiersen', 'the wire - portrait versionyann tiersen', 'thinking like a mountain - portrait versionyann tiersen', 'monochrome - portrait versionyann tiersen', 'gwennilied - portrait versionyann tiersen', 'prad - portrait versionyann tiersen', 'diouz an noz - portrait versionyann tiersen', 'pell - portrait versionyann tiersen', 'tuyo (narcos theme) - a netflix original series soundtrackrodrigo amarante', 'summertime sadness [lana del rey vs. cedric gervais] - cedric gervais remixlana del rey', 'blood / / watergrandson', 'came up (feat. post malone & key!)flosstradamus', 'one day / reckoning song (wankelmut remix) [radio edit]asaf avidan']

diff_spotify_keys = list(diff_spotify.keys())
for i in range(0, len(diff_spotify_keys)):
    if diff_spotify_keys[i] in confirmed_spotify_ids:
        continue
    print(str(i) + ':\t' + diff_spotify[diff_spotify_keys[i]] + '')

0:	o tempo é agora (anavitória) 
	outrória (anavitória)
2:	so am i (george gershwin) 
	i was so young, and you were so beautiful (bill charlap)
3:	(guitar) (alt-j) 
	while my guitar gently weeps (feat. george harrison, paul shaffer & jeff lynne) - alternate mix (the jeff healey band)
39:	up we go (lights) 
	class of 2017 mash-up: my wish / i hope you dance / the climb / i lived (anthem lights)
59:	mountain song (n-so) 
	rocky mountain rangers (colter wall)


In [120]:
# confirmed_genius = [8, 25, 26, 29, 30, 32, 89, 98, 108, 319, 323, 362, 505, 512]
# confirmed_genius_ids = []
# for i in confirmed_genius:
#     confirmed_genius_ids.append(diff_genius_keys[i])
confirmed_genius_ids = ['the other side (from trolls world tour) - oliver heldens remixsza', 'what a wonderful world - one world: together at homecamila cabello', 'stand by me - one world: together at homejohn legend', 'the bones - one world: together at homehozier', 'for you - one world: together at homerita ora', "somewhere over the rainbow_what a wonderful worldisrael kamakawiwo'ole", "smokestack lightnin'howlin' wolf", 'champagne & reefer - livemuddy waters', 'one day / reckoning song (wankelmut remix) - radio editasaf avidan', 'bridged by a lightwave - radio editdeadmau5', 'paper thin - brooks remixillenium', 'despacito - remixluis fonsi', 'blood / / watergrandson', 'one day / reckoning song (wankelmut remix) [radio edit]asaf avidan']

diff_genius_keys = list(diff_genius.keys())
for i in range(0, len(diff_genius_keys)):
    if diff_genius_keys[i] in confirmed_genius_ids:
        continue
    print(str(i) + ':\t' + diff_genius[diff_genius_keys[i]] + '\n\n')

0:	come together - remastered 2009 (the beatles) 
	queen mab (percy bysshe shelley)


1:	motivation - recorded at electric lady studios nyc (muna) 
	new music friday 11/15/19 (spotify)


2:	ana julia (alex cohen) 
	alabama class of 25 (the reffries)


3:	ana júlia (time machine) 
	november 2020 singles release calendar (genius)


4:	ana julia (axe ba) 
	sem palavras (509-e)


5:	prosperity (bootleg rascal) 
	my rap cd collection (platanopeddler)


6:	let it be - remastered 2015 (the beatles) 
	2018 music log (​jeffersuhn)


7:	something - remastered 2009 (the beatles) 
	scp series (the scp foundation)


9:	the smell of rain (two for larry) 
	ulysses (chap. 18 - penelope) (james joyce)


10:	here comes the sun - remastered 2009 (the beatles) 
	dizaster vs. soul khan (king of the dot)


11:	soothe (alan ellis) 
	finnegan's wake (chap. 2.3) (james joyce)


12:	deixe me ir - acústico (1kilo) 
	as 15 melhores lovesongs do rap nacional (genius brasil)


13:	quero ser feliz também - ao vivo (

### Remove Wrong from File

In [121]:
#my_dict.pop('key', None)

# Spotify
for i in range(0, len(diff_spotify_keys)):
    if diff_spotify_keys[i] in confirmed_spotify_ids:
        continue
    ids[diff_spotify_keys[i]].pop('spotifyID', None)
    ids[diff_spotify_keys[i]].pop('spotifyName', None)
    ids[diff_spotify_keys[i]].pop('spotifyArtistName', None)
    
# Genius
for i in range(0, len(diff_genius_keys)):
    if diff_genius_keys[i] in confirmed_genius_ids:
        continue
    ids[diff_genius_keys[i]].pop('geniusID', None)
    ids[diff_genius_keys[i]].pop('geniusName', None)
    ids[diff_genius_keys[i]].pop('geniusArtistName', None)

### Final Count

In [122]:
total_count = len(ids)
spotify_count = 0
genius_count = 0
lyrics_count = 0

for music_id in ids:
    genius_id = ids[music_id].get('geniusID', '-')
    spotify_id = ids[music_id].get('spotifyID', '-')
    
    # count ocurrences
    if genius_id != '-':
        genius_count = genius_count + 1
        if genius_id in lyrics:
            lyrics_count = lyrics_count + 1
    if spotify_id != '-':
        spotify_count = spotify_count + 1
        
print('Total of Songs: ' + str(total_count))
print('Songs that were found on Spotify: ' + str(spotify_count))
print('Songs that were found on Genius: ' + str(genius_count))
print('Songs that have lyrics: ' + str(lyrics_count))

Total of Songs: 2748
Songs that were found on Spotify: 2613
Songs that were found on Genius: 1491
Songs that have lyrics: 1491


### Save new Files

In [125]:
ids_file = open('./Datasets/ids_json_updated.json', 'w', encoding="utf8")
ids_file.write(json.dumps(ids))
ids_file.close()

In [2]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials

birdy_uri = 'spotify:artist:2WX2uTcsvV5OnS0inACecP'
spotify = spotipy.Spotify(client_credentials_manager=SpotifyClientCredentials())

SpotifyOauthError: No client_id. Pass it or set a SPOTIPY_CLIENT_ID environment variable.